<h1> DYNAMIC MALWARE CLASSIFICATION</h1>

Supervised, classification, multi-class

# 0. Basic Set-Up and General info

Note: there are a lot of non-used imports (to be removed at the end)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import seaborn as sns
from scipy import stats

from scipy.stats import loguniform

from skopt.space import Real


import copy

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier


In [3]:
from imblearn.over_sampling import SMOTENC
from boruta import BorutaPy

In [4]:
from library.utils.ownModels.majorityClassModel import MajorityClassClassifier # Importing self-developed model object

In [5]:
import importlib

# Import your modules initially
import library.pipeline

# Reload the modules in case they were updated
importlib.reload(library.pipeline)

# Now you can import specific classes/functions
from library.pipeline.pipeline import Pipeline
from library.pipeline.pipeline_manager import PipelineManager

## Documentation Standarization
Follow the following guidelines to make sure our document is consistent and easier to undestand
- All constant should be writte in full capitall letters (e.g: MY_CONSTANT). Notebook-level constants should be written in the cell assigned to it (see below)
- Every new major section (EDA, Feature engineering) should be written with '# My title'#
- Subsequent sections should use '##', '###' or '####' hierarchacically
- At the beginning of each major section write a content index with the content included in that section (see 'feature engineering' section as an example)
- Please write paragraphs before and after each cell explain what u are about to do and the conclusions, correspondingly. Do not assume they are too obvious.
- Avoid excessive ChatGPT-originated comments
- Avoid writing more than 20 lines per code cell (exceptions for subroutines, which should be written in utilitied_functions.py)
- Ideally, add a "Questions" and "Things to be done" section in each major sections where u write about futher iterations u want to do (while sharing in with the rest)


## Set-up

In [6]:
dataset_path ="./dataset/dynamic_dataset.csv"
results_path = "results/model_evaluation/results.csv"

We start off with a pipeline for all the models. Then at some moment, whenever we think the pipelines will diverge, we remove the by-object-refrence (that we will create in the next cell), and create a new copy with our modifications ready.

## Pipeline Selection
We will having pipelines for:
- ensembled_models (random forest, gradient boosting)
- tree-based (decision trees)
- support vectors machines (non-linear*)
- naive bayes (gaussian naive bayes)
- stacking (stacking models)


* We originally conceived to add linear for the sake of model diversity. While the theory already pointed us out that the linear SVM wont be able to find the separating hyperplane (i.e., model is able to fit), we wanted to see if by setting

In [7]:
default_pipeline = ensembled_pipeline = tree_pipeline = supportVectorsMachine_pipeline = baseline_pipeline = naiveBayes_pipeline = stacking_pipeline = feedForwardNN_pipeline = example = Pipeline(
                        dataset_path=dataset_path, 
                        results_path=results_path,
                        model_type="classification")

In [8]:
pipelines = {
            "not-baseline": {
                  "ensembled": ensembled_pipeline,
                  "tree-based": tree_pipeline,
                  "linear": supportVectorsMachine_pipeline,
                  "naive-bayes": naiveBayes_pipeline,
                  "stacking": stacking_pipeline,
                  }, 
            "baseline": {
                  "baselines": baseline_pipeline, 
                  "example":  example               
            }
}
pipeline_manager = PipelineManager(pipelines)

Here I show an example of how we will be working with by-object-reference.

In [9]:
baseline_pipeline.dataset.example_attribute = "1"

In [ ]:
default_pipeline.dataset.example_attribute

However, if we do a (deep) copy:
Note that the difference between a deep and a shallow copy is that the former copies all the objects that are inside the class. We need that.
We will overwrite the prior attribute and show is this is not propagated to the other results anymore

In [11]:
new_pipeline = copy.deepcopy(default_pipeline)
new_pipeline.dataset.example_attribute = "2"

In [ ]:
default_pipeline.dataset.example_attribute

It may also be of interest for you to propagate the same method's invokation to all the pipelines within the pipeline manager. For instance, after a divergence in, lets say, feature scaling, you may need to fit all the models in the corresponding pipelines. You would want to avoid reduntandly calling the same method with the same parameters for all the Pipeline class instances. In order to do so, we present an example that avoids such annoyance

In [ ]:
pipeline_manager.create_pipeline_divergence(category="baseline", pipelineName="example", print_results=True)

In [ ]:
pipeline_manager.pipelines["baseline"]["example"].feature_analysis.feature_selection.automatic_feature_selection.speak("Whats goooddd")

In [ ]:
pipeline_manager.all_pipelines_execute(methodName="speak", message="Hello, world!")

A method present deeper down the class:

Instead of doing:
'pipeline_manager.pipelines["baseline"]["logistic"].feature_analysis.feature_selection.automatic_feature_selection.speak("Whats goooddd") for all objects'

we can do:

In [ ]:
pipeline_manager.all_pipelines_execute(methodName="feature_analysis.feature_selection.automatic_feature_selection.speak", message="Whats good")

In [17]:
# lets delete the example obj
del new_pipeline 
del pipeline_manager.pipelines["baseline"]["example"]

In [ ]:
pipeline_manager.pipelines

<hr>

# Start Of The Pipeline

In [ ]:
default_pipeline.dataset.df.head()

In [ ]:
for col in default_pipeline.dataset.df.columns.to_list():
    print(col)

In [ ]:
default_pipeline.dataset.df["Category"].value_counts()

In [22]:
default_pipeline.dataset.df.rename(columns={"reboot": "Reboot"}, inplace=True) # consistency with other features' names
default_pipeline.dataset.df.drop(columns=["Family", "Hash"], inplace=True) # We have decided to use only category as target variable; Hash is temporary while im debugging (it will be deleted in EDA)

### Description of our features

For each of our entries in the dataset we have a feature describing the state of the (operating) system under which the malware was running. Here is an intial description for each of them.
Please access this document for a complete problem context description: https://docs.google.com/document/d/1yH9gvnJVSH9GLv9ATQ5JQWA2z8Jy4umxxRfMF-y2fiU/edit?usp=sharing

# 1. EDA

This section below shall be deleted

In [ ]:
default_pipeline.data_preprocessing.remove_duplicates()

In [ ]:
default_pipeline.data_preprocessing.get_missing_values()

In this section, we conduct an Exploratory Data Analysis (EDA) on the dynamic malware dataset to gain initial insights into the structure, distribution, and quality of the data prior to modeling. The dataset includes behavioral features extracted from Android applications, along with labels indicating their respective malware categories. Understanding the composition of the dataset, such as class imbalance, feature correlations, and the presence of outliers, is crucial to ensure robust preprocessing, informed feature engineering, and  the success of machine learning classifiers.


### SUGGESTIONS FOR IMPROVEMENTS
- Cluster analysis

<hr>

# 2. DATA SPLITTING
### TO BE DONE
- Statistical analysis of this
- Make sure they follow the same distributions

### Data Splitting: Category as target variable
Originally, we will focus only on category

Lets first get the X and y extracted from our dataset

Also object!
Lets get back to the splitting!

Before, we split the dataset lets observe the SE of accuracy variation based on our choice of split. 
Brief explanation: we can model accuracy via a Binomial distribution. We know each event in a binomial distribution can be modelled through a bernoulli distribution, where the outcome represents the probability predicting the correct class or not. We make the assumption that each classification error is independent from each other. For:
$$
\text{Bin} \sim (n, p)
$$
 let us assume that the parameter of this distribution is p = .85 and n is given by the choice of sample split for the test set. The SE of the sample proportion can be modeled via: 
$$
\text{SE}_{\hat{p}} = \sqrt{\frac{p(1 - p)}{n}}

$$

Before we continue to assess all possible choices of split based on a variant n, also note that that choice of evenly distributed split (e.g: 10% for each hold-out set) between hold-out sets is arbitrary. Proper choice is that which guarantees an equivalent distribution at each hold-out sets, which may not neccesarily be the equivalent split. This brings a new choice of tradeoff between certainity of prediction accuracy (higher test size, smaller validaiton set) but possibly less space for proper hyperparemter optimization or the inverse (higher validation size, smaller set set). As with other tradeoffs, the priority for a given option is rooted in the model's application (which may come derived from a client/employer). For our case, we dont favor either option of the tradeoof, thus we will keep the even hold-out distribution.

Finally, also note that that choice of evenly distributed split between hold-out sets is arbitrary. Proper choice is that which guarantees an equivalent distribution at each hold-out sets, which may not neccesarily be the equivalent split. This brings a new choice of tradeoff between certainity of prediction accuracy (higher test size, smaller validaiton set) but possibly less space for proper hyperparemter optimization or the inverse (higher validation size, smaller set set). As with other tradeoffs, the priority for a given option is rooted in the model's application (which may come derived from a client/employer). For our case, we dont favor either option of the tradeoof, thus we will keep the even hold-out distribution.


In [ ]:
default_pipeline.dataset.split.asses_split_classifier(p=.85, step=.05, plot=True)

[We can see a diminishing-returns class graph](https://en.wikipedia.org/wiki/Knee_of_a_curve). The more we decline the training set percentage the slower and more steadier the current SE varies as well as the difference to prior SE, in percentage. We can see that the knee in the curve is between 80 and 90 training set percentage. This represents the area where when you start going below the lower bound, no **significant** improve appears. Given the fact that our criteria for choice of split percentage is to keep as much training possible while increasing hold-out sets size only if the decrease in SE is significant **we are going to select 80% for the training set**.

In [26]:
default_pipeline.dataset.split.split_data(y_column="Category",
                                   train_size=.8, 
                                   validation_size=.1,
                                   test_size=.1, 
                                   plot_distribution=False)

In [ ]:
default_pipeline.dataset.X_train.shape, default_pipeline.dataset.X_val.shape, default_pipeline.dataset.X_test.shape, default_pipeline.dataset.y_train.shape, default_pipeline.dataset.y_val.shape, default_pipeline.dataset.y_test.shape

<hr>

# 3. DATA PREPROCESSING

<hr>

This section below shall be properly written. It is set now for early debugging

## !> Pipeline example
Let us show an example of the pipeline divergence. We will create a divergence for the baseline pipeline. We will exempt it from being scaled. This specific example is not meant to be kept, but rather show the purpose of the corresponding functions.

In [40]:
baseline_pipeline = pipeline_manager.create_pipeline_divergence(category="baseline", pipelineName="baselines")

In [ ]:
default_pipeline.dataset.X_train.describe()

In [42]:
default_pipeline.data_preprocessing.scale_features(scaler="robust",
                                      columnsToScale=default_pipeline.dataset.X_train.select_dtypes(include=["number"]).columns)

In [ ]:
default_pipeline.dataset.X_train.describe()

In [ ]:
default_pipeline.dataset.X_train.describe()

In [ ]:
# This is not updated (as expected)
baseline_pipeline.dataset.X_train.describe()

### Outliers 

In [46]:
#models_pipeline.preprocessing.get_outliers_df(plot=True)

<hr>

# 4. FEATURE ANALYSIS

Here we will take adjust the features that compose the learning inputs to our model. The correctness of this section is pivotal for proper learning by the model


## FEATURE ENGINEERING
- Domain-specific features
- Binning
- Interaction terms


In [47]:
featuresToEncode = ["Reboot"]

In [ ]:
encoded_maps_perPipeline = pipeline_manager.all_pipelines_execute(methodName="feature_analysis.feature_transformation.get_categorical_features_encoded", verbose=True, features=featuresToEncode, encode_y=True)
encoded_maps_perPipeline

In [49]:
pipeline_manager.pipelines_analysis.encoded_map = encoded_maps_perPipeline["baseline"]["baselines"] # setting baseline as an arbitrary example (all are the same as for now)

Lets visualize the results of the encoding...

In [ ]:
default_pipeline.dataset.X_train

In [ ]:
default_pipeline.dataset.y_train

In [ ]:
baseline_pipeline.dataset.X_train

In [ ]:
baseline_pipeline.dataset.y_train

## FEATURE SELECTION
- Analyze correlation and low-variances


## Feature selection 
As explained before, we now proceed to carefully reduce the present high-dimensionality. High-dimensionality increases the chances of the model overfitting (capturing noise from irrelevant features; increasing variance and reducing bias), as well as introducing a significant computational overhead. We match this high-dimensionality with a highly filtering models_pipeline of feature selection.
The most extensive cut comes given at the first level, with the mutual information threshold-based cut. This metric captures the level of uncertainity between the feature and the target variable (cnt). In marked contrast with pearson coefficeint (correlation), it is able to model non-linear and linea relationships altoghther. 
This feature-selectin models_pipeline is compromised of five (3 as of the final models_pipeline) cuts:
 - mutual information
 - low variance
 - multicolinearity analysis
 - PCA
 - Boruta and/or Lasso

The different thresholds for each of this cuts have been altered over the different models_pipeline iterations. Specifically, in the bias-variance tradeoff (to be elaborated in further detail later), I increased all the thresholds in order to avoid por performance.

#### Feature to target variable mutual information

In [54]:
# models_pipeline.feature_analysis.feature_selection.manual_feature_selection.fit(type="MutualInformation", threshold=.2, delete_features=True, plot=True)

#### Eliminating low-variances features
Features with low variances provide little new information for the model to learn from, thus they could introduce statistical noise. Due to this reason, they should be elimanted from the dataset. The reason why we don't focus on high-variance is because this symbolizes outliers, which have been dealt with before in data preporcesing.
This function call eliminate low-variance (based on threshold) and all cosntant variables (regardless of threshold)

We will start off this analysis eliminating univariate features (i.e: the featuers with constant values)

In [55]:
# default_pipeline.feature_analysis.feature_selection.manual_feature_selection.fit(type="LowVariances", 
#                                                                          threshold=0.5, 
#                                                                          delete_features=True, 
#                                                                          plot=True)

As explained in scikit-learn's in (1.13.1 Feature selection)[https://scikit-learn.org/stable/modules/feature_selection.html], the variance threshold must be selected carefully. Too low may delete few variables, and too may be too restrictive, deleting more variables than it should.

#### Eliminating highly correlated feature 
Highly correlated variables (multicolinearity) are problem for models because they introduce a redundancy (features that contain significantly related similar information are not bringing much new insight into the model's input) to the model that can introduce significant variance. This is due to the fact that small changes in the data may make the coefficeints of the highly correlated variables **swing** more than it should

A note on the shape of this heatmap: due to the high amount of features, and the redundancy to measure the correlation between features (where corr(A,B) = corr(B,A)) we set 'np.triu(np.ones_like(corr, dtype=bool))' in the utilities functions in order to show only new non-redudant correlations between features, thus the right triangle shape.
Well, thats a lot to digest! We can see some solid red (high positive correlation) and medium-solid blue (some high negative correlation). 
Lets use a non-visual methodology to confirm our initial hypothesis. We will use variance inflaction factor (VIF) along with checking manually.
A brief explanation on VIF. 
Formula: 
$$
VIF_i = \frac{1}{1 - R_i^2}
$$
You regress (i.e: do linear regression) on the ith feature as target variable, and all other features as predictors. You then compute the coefficient of determination as a way to measure how well the predictors fit the target variable. VIF values ranges from [1, inf], where lower bound signifies little multicolinearity (R^2 = 0), and upper bound occurs when R^2=1 (perfect multicolineairty). 5 is considered a standard threshold for VIF as it symbolized an 80% R^2.
Once you ve obtained the results of VIF, you need to delete the variable with the highest VIF, and recompute VIF until there is no multicolinearity. For example, say there are three features that are 4 features, 3 of them being linear combinations of each other. You would delete the variables with VIF until there is no VIF (when only one of those linear combinations remains). You cant delete all n-1 variables where n are the amount of variables with exceeding (with respect to the chosen threshold), because you may be deleting one that has high VIF due to another feature.

A relevant note on why one-hot-encoding must be done dropping the first one:
If we were to not remove one of the labels of one hot encoding, you would be able to predict which level of the categorical variable based on all other ones (there is only degree of freedom for categories levels; they are a linear combination). You would essentially see inf VIF in that area and delete it in this section. 
The VIF has to be computed every time we delete a feature due to high multicolinearity. Lets do that

In [56]:
#models_pipeline.feature_analysis.feature_selection.manual_feature_selection.fit(type="VIF", threshold=10, delete_features=True, plot=False)

### PCA
PCA can still bring some more value to feature selection. It will take into account interaction effects by itself and find the principals that capture as much variance as we specify. Thus, its inclusion in the feature selection models_pipeline.
It has been excldued due to underperformance.

In [57]:
#models_pipeline.feature_analysis.feature_selection.manual_feature_selection.fit(type="PCA", threshold=.95, delete_features=False, plot=False)

### 1.3 Automatic Feature Selection
#### L1 regularization
Because of L1 regularization being able to set weights 0, we can briefly train our logistic regression model with such regulartization and see which features it uses. The reason why it sets 0 to insignificant feature is because the objective function is not only the MSE but added a component of the wieght magnitude (which is trying to minimize)

In [58]:
# excluded_features, predictive_features, coefficients = models_pipeline.feature_analysis.feature_selection.automatic_feature_selection.fit(type="L1",
#                                                                                                                                     max_iter=1000, 
#                                                                                                                                     print_results=True, delete_features=False)
# excluded_features, len(excluded_features)

#### BORUTA
Boruta is a more powerful feature selection method (thus we use it as a reference for variable deletion). It is more powerful that L1 becuase it compares the importance of features to shuffled versions, ensuring robust feature selection


In [59]:
# excluded_features, predictive_features = models_pipeline.feature_analysis.feature_selection.automatic_feature_selection.fit(type="Boruta", max_iter=10, print_results=True, delete_features=False)
# excluded_features, len(excluded_features)

Awesome, lets move onto the actual modelling part!

<hr>

# MODELLING

## Fitting the model

QUESTIONS FOR THIS SECTION
- Can the ROC curve be used for multiclass?
- Can a unsupervised learning algorithm (e.g: KNN) be used for this problem even tough its nature is to be supervised?
- Does val to train deltas are meaningful here?

TO BE DONE FOR THIS SECTION
- Multiple models as classifiers
- Learn more about each alogrithms' paremters
- Can KNN be used here?

## Random Forest & Decision Trees
Instead of the originally planned logistic regression, we will be using an ensembled model first: random forest (collection of week decision trees). This model is not only likely to outperform the original choice because its nature to handle multiclass better, but also does this several orders of magnitude faster. We will add its not ensembled version too, along with gradient boosted machine
Note we are not using not-by-default multiclass classifiers (e.g: logistic regressions, svms)

## Non-optimized fitting
We first fit all these models with the default paramterers. This is done to constrat more starkly the difference between pre and post tuning.

In [60]:
# Ensembled models
gradientBoostingModel = GradientBoostingClassifier()
randomForestModel = RandomForestClassifier()

# Tree-based models
decisionTreeModel = DecisionTreeClassifier()

# Support Vector Machines models
nonLinearSupportVectorModel = SVC()
linearSupportVectorModel = LinearSVC()

# Naive Bayes model
naiveBayesModel = GaussianNB()

# Stacking model
stackingModel = StackingClassifier(
      estimators=[
            ("Random Forest", randomForestModel),
            ("Decision Tree", decisionTreeModel),
            ("Non-Linear SVM", nonLinearSupportVectorModel),
            ("Linear SVM", linearSupportVectorModel),
            ("Gaussian Naive Bayes", naiveBayesModel)
      ],
      final_estimator=DecisionTreeClassifier(),
      cv=5,
      verbose=3
)

# Baseline models
logisticRegressionModel = LogisticRegression()
majorityClassModel = MajorityClassClassifier()

#### MODEL PERFORMANCE, HYPOTHESIS
- Trees (e.g: random forest, decision trees):
  - Time to fit:
    - Ensemble models (random forest) take more time to train due to the fact that they are larger and heavier than their non-ensembled version. 
  - Correctness:
     - High
- Binary-classifiers by default models (e.g: SMV, logistic regression)
  - Time to fit:
    - compute C models for all C number of classes. Each trained to detect a single class, then when we make predictions, we select the ones that has the highest probability in its predictions ("the most confident in its prediction"). This strategy is called One-vs-Rest, note however, a single logistic regression may be used if we used the softmax objective function (instead of log-odds) (it still heavy computationally, tough). They will be sloder than ensemble models
  - Correctness:
     - Very low if the problem is non-linear which it is for the SVM and logistic regression

In [ ]:
pipeline_manager.pipelines

Pipelines always need to diverge from training onwards. Otherwise they will have each other results (which does not follow the isolation pattern we have programmed this with)

In [62]:
ensembled_pipeline = pipeline_manager.create_pipeline_divergence(category="not-baseline", pipelineName="ensembled")
supportVectorsMachine_pipeline = pipeline_manager.create_pipeline_divergence(category="not-baseline", pipelineName="linear")
naiveBayes_pipeline = pipeline_manager.create_pipeline_divergence(category="not-baseline", pipelineName="naive-bayes")
stacking_pipeline = pipeline_manager.create_pipeline_divergence(category="not-baseline", pipelineName="stacking")

In [ ]:
pipeline_manager.pipelines

In [64]:
# Ensembled models
ensembled_pipeline.model_selection.add_model("Gradient Boosting", gradientBoostingModel)
ensembled_pipeline.model_selection.add_model("Random Forest", randomForestModel)

# Tree-based models
tree_pipeline.model_selection.add_model("Decision Tree", decisionTreeModel)

# Support Vector Machines models
supportVectorsMachine_pipeline.model_selection.add_model("Non-Linear SVM", nonLinearSupportVectorModel) 
supportVectorsMachine_pipeline.model_selection.add_model("Linear SVM", linearSupportVectorModel) 

# Naive Bayes
naiveBayes_pipeline.model_selection.add_model("Gaussian Naive Bayes", naiveBayesModel)

# Stacking
stacking_pipeline.model_selection.add_model("Stacking", stackingModel)

# Baseline
baseline_pipeline.model_selection.add_model("Logistic Regression (baseline)", logisticRegressionModel)
baseline_pipeline.model_selection.add_model("Majority Class (baseline)", majorityClassModel)


#### THIS WILL BE DELETED
While we debug, lets exlclude some models we dont need for now (they are very slow to train).

#### THIS WONT BE DELETED
As indicated throughout the notebook this is the part where we eliminate the models that have been excluded from the final training.

| Model | Reason for Exclusion |
|-------|----------------------|
|Linear SVM | Ran the training for 1h never converged| 

In [65]:
# Ensembled models
ensembled_pipeline.model_selection.models_to_exclude = []

# Tree-based models
tree_pipeline.model_selection.models_to_exclude = []

# Support Vector Machines models
supportVectorsMachine_pipeline.model_selection.models_to_exclude = ["Linear SVM"]

# Naive Bayes models
naiveBayes_pipeline.model_selection.models_to_exclude = []

# Baseline models
baseline_pipeline.model_selection.models_to_exclude = []


Currently the stack model has as base predictos all models from all the pipelines. We would now need to update them based on the prior decisions to exclude from the training certain models. This is under the assumption that if a given base model it is excluded from its individual training it is not intended to be triggered an indivual training when the meta model (from the stack model) spots it is untrained.
Lets make sure they are deleted!:

In [ ]:
estimators = []

for index, estimator in enumerate(stackingModel.estimators):
      estimators.append(estimator[0])
estimators

n_of_estimators = len(stackingModel.estimators)

for pipelineName, pipeline in pipeline_manager.pipelines["not-baseline"].items():
      for modelToExclude in pipeline.model_selection.models_to_exclude:
            if modelToExclude in estimators:
                  for index, estimator in enumerate(stackingModel.estimators):
                        if estimator[0] == modelToExclude:
                              stackingModel.estimators.pop(index)

stackingModel.estimators


Lets finally start fitting the models:

In [67]:
pipeline_manager.pipeline_state = "pre"

In [ ]:
pipeline_manager.all_pipelines_execute(methodName="model_selection.fit_models",
                                       verbose=False, 
                                       exclude_pipeline_names=["stacking"],
                                       current_phase=pipeline_manager.pipeline_state)

Let's make sure the predictions vary between holdout sets

<i> the aforeshown diagram was originally done with the sole intention to debug an error that made predictions be the same across sets. Insights may not be much meaningful after correctio, but it is worth keeping until the end of the notebook development </i>

### PREDICTIONS RESULTS 
Before we get into the actual results, lets elaborate briefly on all the metrics that we are using to asses our classifiers:

- Accuracy => total correctly predicted elemetnts (sigma over the moments we predicted x_i and it was actually x_i / number_of_samples)
$$
\text{Accuracy} = \frac{\sum_{i} \mathbf{1}(\hat{y}_i = y_i)}{N}
$$
- Precision => out of how many predicted for that class were actually from that class (predicted for class x when it was x/ predicted for class x when it was x + predicted for class x when it was NOT x)
$$
\text{Precision}_x = \frac{\text{TP}_x}{\text{TP}_x + \text{FP}_x}
$$
- Recall => out of all cases that were positive how many got predicted correctly?
$$
\text{Recall}_x = \frac{\text{TP}_x}{\text{TP}_x + \text{FN}_x}
$$
- F1-score => harmonic mean of precision and recall (balances both metrics, heavily penalize spreadness between ratios that are being averaged out)
$$
\text{F1}_x = 2 \times \frac{\text{Precision}_x \times \text{Recall}_x}{\text{Precision}_x + \text{Recall}_x}
$$
- Support => number of actual occurences of class in the dataset
- macro avg => averages given metric across all classes
$$
\text{Macro Avg} = \frac{1}{C} \sum_{i=1}^{C} M_i
$$
- weighted avg => averages with weights per class occurence (considers frequency of class in average computation)
$$
\text{Weighted Avg} = \sum_{i=1}^{C} \frac{\text{Support}_i}{\text{Total Instances}} \times M_i
$$

In [69]:
comments = "wiLL THIS work?"

In [ ]:
pipeline_manager.pipelines["baseline"]["baselines"].model_selection.list_of_models["Logistic Regression (baseline)"].tuning_states["pre"].assesment

In [ ]:
pipeline_manager.pipelines["baseline"]["baselines"].model_selection.list_of_models["Majority Class (baseline)"].tuning_states["pre"].assesment

In [ ]:
pipeline_manager.pipelines["baseline"]["baselines"].model_selection.list_of_models["Majority Class (baseline)"].tuning_states["pre"].assesment["model_sklearn"].get_params()

In [ ]:
model_results = pipeline_manager.all_pipelines_execute(methodName="model_selection.evaluate_and_store_models", 
                                                       verbose=False,
                                                       exclude_pipeline_names=["stacking"],
                                                       comments=comments, 
                                                       current_phase=pipeline_manager.pipeline_state)

## Performance Evaluation 
Below are shown all the metrics we can compare our plots to:

### Cross-Model Evaluation (pre-tuning)

In [ ]:
pipeline_manager.pipelines_analysis._compute_classification_report()

In [ ]:
pipeline_manager.pipelines_analysis.plot_cross_model_comparison(metric=["f1-score", "recall", "precision"])


In [ ]:
metrics_df = pipeline_manager.pipelines_analysis.plot_results_df(metrics=["timeToFit", "timeToPredict"])
metrics_df

### Results Summary Visualization

In [ ]:
pipeline_manager.pipelines_analysis.plot_results_summary(training_metric="timeToFit", performance_metric="accuracy")

### Intra-model Evaluation (pre-tuning)
We plot 3 metrics (f1, recall and precision) for each model constrasting its performance between trainign and validation. It is not subject of interest cross-model evaluation, but rather, intra-model evaluation.

In [ ]:
pipeline_manager.pipelines_analysis.plot_intra_model_comparison(metrics=["f1-score", "recall", "precision"])

### Residual analysis (pre-tuning)

This contains the confusion matrices (weighted and not weighted). 
It also returns the specific elements that were erroneously classified. The analysis of this residuals may be meaningful for further inspection (clustering analysis, EDA...)

In [ ]:
residuals, confusion_matrices = pipeline_manager.pipelines_analysis.plot_confusion_matrix()
residuals

### Feature importances (pre-tuning)

In [ ]:
importances_dfs = pipeline_manager.pipelines_analysis.plot_feature_importance()
importances_dfs

## Hyperparameter Optimization
1. Bind the grids to each model in each pipeline
2. Start all optimizations in parallel

In [81]:
pipeline_manager.pipeline_state = "in"

In [82]:
rf_grid = {
      'n_estimators': [50, 100, 150, 200], 
      'max_depth': [None, 10, 20, 30], 
      'min_samples_split': [2, 5, 10], 
      'min_samples_leaf': [1, 2, 4,]
}

dt_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': [None, 'sqrt', 'log2'],
    'ccp_alpha': [0.0, 0.01, 0.1]

} 

gnb_grid = {
    'var_smoothing': Real(1e-12, 1e-6, prior='log-uniform')
}


In [83]:
modelNameToOptimizer = {
      "Random Forest": {
            "optimizer_type": "bayes",
            "param_grid": rf_grid,
            "max_iter": 5
      },
      "Decision Tree": {
            "optimizer_type": "bayes",
            "param_grid": dt_grid,
            "max_iter": 5
      },
      "Gaussian Naive Bayes": {
        "optimizer_type": "bayes",
        "param_grid": gnb_grid,
        "max_iter": 5
      }
      }

In [ ]:
pipeline_manager.all_pipelines_execute(methodName="model_selection.fit_models", 
                                       current_phase=pipeline_manager.pipeline_state,
                                       modelNameToOptimizer=modelNameToOptimizer)

## Stacking Model: Training and Hyperparamter Optimization

In [85]:
stacking_grid = {
    'final_estimator__max_depth': [10, 20, 30],
    'final_estimator__min_samples_split': [2, 5, 10],
    'final_estimator__min_samples_leaf': [1, 2, 4],
    'final_estimator__max_features': [None, 'sqrt', 'log2'],
    'final_estimator__ccp_alpha': [0.0, 0.01, 0.1],
    'stack_method': ['predict', 'predict_proba'],  
    'passthrough': [False, True]  
}

In [86]:
modelNameToOptimizer = {
      "Stacking": {
            "optimizer_type": "bayes",
            "param_grid":stacking_grid,
            "max_iter": 5
      }
}

In [ ]:
pipeline_manager.all_pipelines_execute(methodName="model_selection.fit_models", 
                                       current_phase=pipeline_manager.pipeline_state,
                                       modelNameToOptimizer=modelNameToOptimizer)

## In-tuning evaluation

In [ ]:
model_results = pipeline_manager.all_pipelines_execute(methodName="model_selection.evaluate_and_store_models", 
                                                       exclude_categories=["baseline"],
                                                       comments=comments, 
                                                       current_phase=pipeline_manager.pipeline_state)

### Cross-Model Evaluation (in-tuning)

In [ ]:
pipeline_manager.pipelines_analysis._compute_classification_report().T

In [ ]:
pipeline_manager.pipelines_analysis.plot_cross_model_comparison(metric=["f1-score", "recall", "precision"])

In [ ]:
pipeline_manager.pipelines_analysis.plot_results_df(metrics=["timeToFit", "timeToPredict"])
# Note I still have not added a way to extract the fitting time from the in-tuning phase

### Results Summary Visualization

In [ ]:
pipeline_manager.pipelines_analysis.plot_results_summary(training_metric="timeToPredict", performance_metric="accuracy")

### Residual analysis (in-tuning)

This contains the confusion matrices (weighted and not weighted). 
It also returns the specific elements that were erroneously classified. The analysis of this residuals may be meaningful for further inspection (clustering analysis, EDA...)

In [ ]:
residuals, confusion_matrices = pipeline_manager.pipelines_analysis.plot_confusion_matrix()
residuals

### Feature importances (in-tuning)

In [ ]:
importances_dfs = pipeline_manager.pipelines_analysis.plot_feature_importance()
importances_dfs

## Optimizer Covergance
Before we proceed, note that you can access the optimizser object (grid, random or bayes) as per shown in the following example. You may see more detailed information on the tuning process there. For now, we limit ourselves to solely plot the convergence map of the bayes optimizer

Questions:
- what can further be analyzed from this process in order to get better insights?

In [ ]:
pipeline_manager.pipelines["not-baseline"]["ensembled"].model_selection.list_of_models["Random Forest"].tuning_states["in"].assesment # just an example

In [ ]:
pipeline_manager.all_pipelines_execute(methodName="model_selection.plot_convergence")

# Post-tuning Evaluation
Lets start off by selecting the best perfoming model

In [ ]:
best_model, best_score = pipeline_manager.select_best_performing_model(metric="precision")

In [ ]:
pipeline_manager.fit_final_models()

In [ ]:
pipeline_manager.evaluate_store_final_models()

### Metric Evaluation

In [100]:
pipeline_manager.pipeline_state = "post"

### Cross-Model Evaluation (post-tuning)

In [ ]:
class_report_df = pipeline_manager.pipelines_analysis._compute_classification_report(include_training=True)
class_report_df

In [ ]:
pipeline_manager.pipelines_analysis.plot_cross_model_comparison(metric=["f1-score", "recall", "precision"])

In [ ]:
pipeline_manager.pipelines_analysis.plot_results_df(metrics=["timeToFit", "timeToPredict"])


### Results Summary Visualization

In [ ]:
pipeline_manager.pipelines_analysis.plot_results_summary(training_metric="timeToFit", performance_metric="accuracy")

### Intra-model Evaluation (post-tuning)
We plot 3 metrics (f1, recall and precision) for each model constrasting its performance between trainign and validation. It is not subject of interest cross-model evaluation, but rather, intra-model evaluation.

In [ ]:
pipeline_manager.pipelines_analysis.plot_intra_model_comparison(metrics=["f1-score", "recall", "precision"])


### Residual analysis (post-tuning)

This contains the confusion matrices (weighted and not weighted). 
It also returns the specific elements that were erroneously classified. The analysis of this residuals may be meaningful for further inspection (clustering analysis, EDA...)

In [ ]:
residuals, confusion_matrices = pipeline_manager.pipelines_analysis.plot_confusion_matrix()
residuals

### Feature importances (post-tuning)

In [ ]:
importances_dfs = pipeline_manager.pipelines_analysis.plot_feature_importance()
importances_dfs

## Saving Best Model

In [108]:
#pipeline_manager.store_best_performing_sklearn_model_object(includeModelObject=False)

In [109]:
#pipeline_manager.store_pipelines(includeModelObject=False) # this is going to heavy in size
